# contents

1. 일사량 데이터 전처리 + 기상데이터(+발전데이터)데이터셋과 결합
2. 계절 column 만들기
3. 대기데이터 결합 (1-4에서 만듬)  
  
  
  
    
    
- 앞 과정에서 사용했던 함수들이기 때문에 상세 주석 생략함 
  
* 주의 : 대기데이터에 null 값이 있음

# 1. 일사량 추정 데이터 전처리 및 기상+발전량데이터셋과 결합
- 강릉지역 법선면 일사량 데이터(추정치)와 기상+발전 데이터 결합 
    - 데이터 : 법선면 일사량 (추정치), 김민주 원우님 제공
        - 법선면 일사량 (추정치)의 시간은 1~8760으로 되어 있음
        - 키값으로 변경하기 위해서 만들어 놓은 데이터 시간을 가져다 쓰기로 생각함 
        - 발전량 데이터를 불러와서 시간 column을 일사량 데이터로 가져다가 씀

- 작업내용   
    1) 강릉지역 법선면 일사량 (추정치) 데이터 불러오기   
    2) 태양광 발전량 데이터 불러오기   
    3) 데이터 결합하기   
    4) 필요없는 시간 제거하기  
    5) 키값 만들기  
    6) 기상데이터와 결합

In [1]:
import pandas as pd

### 1) 강릉지역 법선면 일사량 (추정치) 데이터 불러오기

In [29]:
# 데이터 불러오기 
gangneung_solar_radiation = pd.read_excel('강릉일사량데이터.xlsx')

In [30]:
# 시간이 1~ 8760까지 숫자로 되어 있음 
# 태양광 데이터의 시간을 붙여서 쓰려고 생각함

gangneung_solar_radiation.tail()

,시간,건구온도 (℃),이슬점온도 (℃),상대습도 (%),현지기압 (hPa),전운량 (1/10),풍속 (m/s),수평면 전일사량 (Wh/m²),법선면 직달일사량 (Wh/m²)
8755,8756,2.2,-0.9,78.0,1027.1,0,4.0,0.0,0.0
8756,8757,1.7,-1.5,78.0,1026.9,0,2.9,0.0,0.0
8757,8758,1.0,-2.9,77.0,1026.7,0,1.7,0.0,0.0
8758,8759,1.3,-4.3,63.0,1026.6,0,3.1,0.0,0.0
8759,8760,1.2,-5.7,59.0,1026.4,4,2.8,0.0,0.0


### 2) 태양광 발전량 데이터 불러오기

In [31]:
power = pd.read_csv('donghae_solar_power2018.csv', index_col=0)

In [32]:
power['시간']=gangneung_solar_radiation['시간']

In [33]:
power.tail()

,time,power,hour,date,year,시간
8755,20시,0,20,2018-12-31,2018,8756
8756,21시,0,21,2018-12-31,2018,8757
8757,22시,0,22,2018-12-31,2018,8758
8758,23시,0,23,2018-12-31,2018,8759
8759,24시,0,24,2018-12-31,2018,8760


### 3) 데이터 결합하기 

In [34]:
df = pd.merge(gangneung_solar_radiation, power)

In [35]:
df.head()

,시간,건구온도 (℃),이슬점온도 (℃),상대습도 (%),현지기압 (hPa),전운량 (1/10),풍속 (m/s),수평면 전일사량 (Wh/m²),법선면 직달일사량 (Wh/m²),time,power,hour,date,year
0,1,-0.5,-4.3333,75.0,1023.4,0,0.3,0.0,0.0,1시,0,1,2018-01-01,2018
1,2,-1.5,-4.7667,79.0,1022.8,0,0.3,0.0,0.0,2시,0,2,2018-01-01,2018
2,3,-1.8,-5.2000,79.0,1022.2,0,0.4,0.0,0.0,3시,0,3,2018-01-01,2018
3,4,-0.3,-5.1333,68.0,1021.5,0,2.7,0.0,0.0,4시,0,4,2018-01-01,2018
4,5,-0.2,-5.0667,69.0,1020.9,0,1.1,0.0,0.0,5시,0,5,2018-01-01,2018


In [36]:
df.columns

Index(['시간', '건구온도 (℃)', '이슬점온도 (℃)', '상대습도 (%)', '현지기압 (hPa)', '전운량 (1/10)',
       '풍속 (m/s)', '수평면 전일사량 (Wh/m²)', '법선면 직달일사량 (Wh/m²)', 'time', 'power',
       'hour', 'date', 'year'],
      dtype='object')

In [37]:
df=df[['date', 'hour', '법선면 직달일사량 (Wh/m²)']]

In [38]:
df.columns = ['date','hour','solar_radiation_nomal']

In [39]:
df.head()

,date,hour,solar_radiation_nomal
0,2018-01-01,1,0.0
1,2018-01-01,2,0.0
2,2018-01-01,3,0.0
3,2018-01-01,4,0.0
4,2018-01-01,5,0.0


In [40]:
df.tail()

,date,hour,solar_radiation_nomal
8755,2018-12-31,20,0.0
8756,2018-12-31,21,0.0
8757,2018-12-31,22,0.0
8758,2018-12-31,23,0.0
8759,2018-12-31,24,0.0


### 4) 필요없는 시간 제거하기 

In [41]:
# 해 없는 시간 제거하기 
result = []
rej_time = [1,2,3,4,5, 21,22,23,24]
for j in rej_time:
    one_indice = []
    for i in range(len(df[df['hour'] ==j].index)):
        one = df[df['hour'] ==j].index[i]
        one_indice.append(one)
        
    result.extend(one_indice)    

result

[0,
 24,
 48,
 72,
 96,
 120,
 144,
 168,
 192,
 216,
 240,
 264,
 288,
 312,
 336,
 360,
 384,
 408,
 432,
 456,
 480,
 504,
 528,
 552,
 576,
 600,
 624,
 648,
 672,
 696,
 720,
 744,
 768,
 792,
 816,
 840,
 864,
 888,
 912,
 936,
 960,
 984,
 1008,
 1032,
 1056,
 1080,
 1104,
 1128,
 1152,
 1176,
 1200,
 1224,
 1248,
 1272,
 1296,
 1320,
 1344,
 1368,
 1392,
 1416,
 1440,
 1464,
 1488,
 1512,
 1536,
 1560,
 1584,
 1608,
 1632,
 1656,
 1680,
 1704,
 1728,
 1752,
 1776,
 1800,
 1824,
 1848,
 1872,
 1896,
 1920,
 1944,
 1968,
 1992,
 2016,
 2040,
 2064,
 2088,
 2112,
 2136,
 2160,
 2184,
 2208,
 2232,
 2256,
 2280,
 2304,
 2328,
 2352,
 2376,
 2400,
 2424,
 2448,
 2472,
 2496,
 2520,
 2544,
 2568,
 2592,
 2616,
 2640,
 2664,
 2688,
 2712,
 2736,
 2760,
 2784,
 2808,
 2832,
 2856,
 2880,
 2904,
 2928,
 2952,
 2976,
 3000,
 3024,
 3048,
 3072,
 3096,
 3120,
 3144,
 3168,
 3192,
 3216,
 3240,
 3264,
 3288,
 3312,
 3336,
 3360,
 3384,
 3408,
 3432,
 3456,
 3480,
 3504,
 3528,
 3552,
 3576

In [42]:
df.drop(result, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,date,hour,solar_radiation_nomal
0,2018-01-01,6,0.0000
1,2018-01-01,7,0.0000
2,2018-01-01,8,0.0000
3,2018-01-01,9,4.7812
4,2018-01-01,10,17.7890


In [43]:
df.tail()

,date,hour,solar_radiation_nomal
5470,2018-12-31,16,430.85
5471,2018-12-31,17,121.58
5472,2018-12-31,18,0.00
5473,2018-12-31,19,0.00
5474,2018-12-31,20,0.00


In [44]:
# 소수점 둘째자리까지 반올림
solars=[]
for i in range(len(df['date'])):
    solar = round(df.solar_radiation_nomal[i], 2)
    solars.append(solar)
    
df['solar'] = solars

In [45]:
df.head()

,date,hour,solar_radiation_nomal,solar
0,2018-01-01,6,0.0000,0.00
1,2018-01-01,7,0.0000,0.00
2,2018-01-01,8,0.0000,0.00
3,2018-01-01,9,4.7812,4.78
4,2018-01-01,10,17.7890,17.79


In [46]:
df = df[['date', 'hour', 'solar']]

In [47]:
df.columns = ['date','hour','solar_radiation_normal']

In [48]:
df.head()

,date,hour,solar_radiation_normal
0,2018-01-01,6,0.00
1,2018-01-01,7,0.00
2,2018-01-01,8,0.00
3,2018-01-01,9,4.78
4,2018-01-01,10,17.79


In [49]:
df.tail()

,date,hour,solar_radiation_normal
5470,2018-12-31,16,430.85
5471,2018-12-31,17,121.58
5472,2018-12-31,18,0.00
5473,2018-12-31,19,0.00
5474,2018-12-31,20,0.00


### 5) 키값 만들기

In [50]:
df['hour'] = df['hour'].astype(str)

In [51]:
# 키값 만들기
df['date_time'] = df['date'] + '-' + df['hour']

In [52]:
df.head()

,date,hour,solar_radiation_normal,date_time
0,2018-01-01,6,0.00,2018-01-01-6
1,2018-01-01,7,0.00,2018-01-01-7
2,2018-01-01,8,0.00,2018-01-01-8
3,2018-01-01,9,4.78,2018-01-01-9
4,2018-01-01,10,17.79,2018-01-01-10


In [53]:
df2= df[['date_time', 'solar_radiation_normal']]

In [54]:
df2.head()

,date_time,solar_radiation_normal
0,2018-01-01-6,0.00
1,2018-01-01-7,0.00
2,2018-01-01-8,0.00
3,2018-01-01-9,4.78
4,2018-01-01-10,17.79


### 6) 기상데이터와 결합

In [55]:
# 만들어놓았던 기상데이터 불러오기
df10 = pd.read_csv('donghae_solar_power_dataset2018.csv', index_col=0)

In [56]:
df10.head()

,date,hour,humidity,rain,rain_type,temp,thunder,wind_angle,wind_velo,sky_con,power,date_time
0,2018-01-01,6,17.0,0.0,0.0,4.7,0.0,289.0,2.8,1,0,2018-01-01-6
1,2018-01-01,7,19.0,0.0,0.0,3.4,0.0,278.0,2.7,1,0,2018-01-01-7
2,2018-01-01,8,14.0,0.0,0.0,2.7,0.0,268.0,2.3,1,7,2018-01-01-8
3,2018-01-01,9,18.0,0.0,0.0,2.3,0.0,268.0,2.3,2,245,2018-01-01-9
4,2018-01-01,10,20.0,0.0,0.0,2.1,0.0,250.0,3.0,2,776,2018-01-01-10


In [57]:
df100 = pd.merge(df10, df2)

In [58]:
df100.head()

,date,hour,humidity,rain,rain_type,temp,thunder,wind_angle,wind_velo,sky_con,power,date_time,solar_radiation_normal
0,2018-01-01,6,17.0,0.0,0.0,4.7,0.0,289.0,2.8,1,0,2018-01-01-6,0.00
1,2018-01-01,7,19.0,0.0,0.0,3.4,0.0,278.0,2.7,1,0,2018-01-01-7,0.00
2,2018-01-01,8,14.0,0.0,0.0,2.7,0.0,268.0,2.3,1,7,2018-01-01-8,0.00
3,2018-01-01,9,18.0,0.0,0.0,2.3,0.0,268.0,2.3,2,245,2018-01-01-9,4.78
4,2018-01-01,10,20.0,0.0,0.0,2.1,0.0,250.0,3.0,2,776,2018-01-01-10,17.79


In [59]:
df100.columns

Index(['date', 'hour', 'humidity', 'rain', 'rain_type', 'temp', 'thunder',
       'wind_angle', 'wind_velo', 'sky_con', 'power', 'date_time',
       'solar_radiation_normal'],
      dtype='object')

In [60]:
df100=df100[['date', 'hour', 'humidity', 'rain', 'rain_type', 'temp', 'thunder',
       'wind_angle', 'wind_velo', 'sky_con', 'solar_radiation_normal','power', 'date_time']]

In [61]:
df100.head()

,date,hour,humidity,rain,rain_type,temp,thunder,wind_angle,wind_velo,sky_con,solar_radiation_normal,power,date_time
0,2018-01-01,6,17.0,0.0,0.0,4.7,0.0,289.0,2.8,1,0.00,0,2018-01-01-6
1,2018-01-01,7,19.0,0.0,0.0,3.4,0.0,278.0,2.7,1,0.00,0,2018-01-01-7
2,2018-01-01,8,14.0,0.0,0.0,2.7,0.0,268.0,2.3,1,0.00,7,2018-01-01-8
3,2018-01-01,9,18.0,0.0,0.0,2.3,0.0,268.0,2.3,2,4.78,245,2018-01-01-9
4,2018-01-01,10,20.0,0.0,0.0,2.1,0.0,250.0,3.0,2,17.79,776,2018-01-01-10


In [62]:
df100[df100['power'] >1000]

,date,hour,humidity,rain,rain_type,temp,thunder,wind_angle,wind_velo,sky_con,solar_radiation_normal,power,date_time
5,2018-01-01,11,26.0,0.0,0.0,2.1,0.0,259.0,3.8,2,291.69,1361,2018-01-01-11
6,2018-01-01,12,27.0,0.0,0.0,2.2,0.0,264.0,3.8,2,353.29,1681,2018-01-01-12
7,2018-01-01,13,27.0,0.0,0.0,1.9,0.0,255.0,3.1,2,179.65,1733,2018-01-01-13
8,2018-01-01,14,29.0,0.0,0.0,1.6,0.0,251.0,2.4,2,46.11,1645,2018-01-01-14
9,2018-01-01,15,28.0,0.0,0.0,2.0,0.0,265.0,3.6,2,188.56,1231,2018-01-01-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5409,2018-12-27,15,20.0,0.0,0.0,-7.0,-1.0,302.0,3.6,1,352.84,1147,2018-12-27-15
5424,2018-12-28,15,19.0,0.0,0.0,-4.9,-1.0,289.0,5.1,1,800.67,1132,2018-12-28-15
5439,2018-12-29,15,23.0,0.0,0.0,-4.0,-1.0,280.0,2.7,1,806.73,1154,2018-12-29-15
5454,2018-12-30,15,46.0,0.0,0.0,-3.5,-1.0,244.0,1.8,1,401.51,1136,2018-12-30-15


In [63]:
# df100.to_csv('donghae_solar_power_dataset2018_rev.csv')

# 2. 계절 만들기
- timestamp로 만들어서 구간별 값 입력 

In [64]:
df100.tail()

,date,hour,humidity,rain,rain_type,temp,thunder,wind_angle,wind_velo,sky_con,solar_radiation_normal,power,date_time
5470,2018-12-31,16,26.0,0.0,0.0,-2.7,-1.0,228.0,1.4,3,430.85,457,2018-12-31-16
5471,2018-12-31,17,18.0,0.0,0.0,-1.2,-1.0,253.0,1.8,2,121.58,117,2018-12-31-17
5472,2018-12-31,18,24.0,0.0,0.0,-2.7,-1.0,217.0,1.8,2,0.00,2,2018-12-31-18
5473,2018-12-31,19,22.0,0.0,0.0,-2.7,-1.0,209.0,1.1,3,0.00,0,2018-12-31-19
5474,2018-12-31,20,23.0,0.0,0.0,-2.7,-1.0,217.0,1.2,3,0.00,0,2018-12-31-20


In [65]:
# timestamp로 바꾸기 
import datetime
date_times=[]
for i in range(len(df100['date'])):
    date_time = datetime.datetime.strptime(df100['date'][i], '%Y-%m-%d')
    date_times.append(date_time)
    
df100['timestamp'] = date_times

In [66]:
df100.head()

,date,hour,humidity,rain,rain_type,temp,thunder,wind_angle,wind_velo,sky_con,solar_radiation_normal,power,date_time,timestamp
0,2018-01-01,6,17.0,0.0,0.0,4.7,0.0,289.0,2.8,1,0.00,0,2018-01-01-6,2018-01-01
1,2018-01-01,7,19.0,0.0,0.0,3.4,0.0,278.0,2.7,1,0.00,0,2018-01-01-7,2018-01-01
2,2018-01-01,8,14.0,0.0,0.0,2.7,0.0,268.0,2.3,1,0.00,7,2018-01-01-8,2018-01-01
3,2018-01-01,9,18.0,0.0,0.0,2.3,0.0,268.0,2.3,2,4.78,245,2018-01-01-9,2018-01-01
4,2018-01-01,10,20.0,0.0,0.0,2.1,0.0,250.0,3.0,2,17.79,776,2018-01-01-10,2018-01-01


In [67]:
df100['season'] = 0
df100.head()

,date,hour,humidity,rain,rain_type,temp,thunder,wind_angle,wind_velo,sky_con,solar_radiation_normal,power,date_time,timestamp,season
0,2018-01-01,6,17.0,0.0,0.0,4.7,0.0,289.0,2.8,1,0.00,0,2018-01-01-6,2018-01-01,0
1,2018-01-01,7,19.0,0.0,0.0,3.4,0.0,278.0,2.7,1,0.00,0,2018-01-01-7,2018-01-01,0
2,2018-01-01,8,14.0,0.0,0.0,2.7,0.0,268.0,2.3,1,0.00,7,2018-01-01-8,2018-01-01,0
3,2018-01-01,9,18.0,0.0,0.0,2.3,0.0,268.0,2.3,2,4.78,245,2018-01-01-9,2018-01-01,0
4,2018-01-01,10,20.0,0.0,0.0,2.1,0.0,250.0,3.0,2,17.79,776,2018-01-01-10,2018-01-01,0


In [68]:
# 2018년 입춘, 입하, 입추 일
sp_indice = df100[(df100['timestamp'] > "2018-02-03") & (df100['timestamp'] < "2018-05-05")].index
sm_indice = df100[(df100['timestamp'] > "2018-05-04") & (df100['timestamp'] < "2018-08-07")].index
fa_indice = df100[(df100['timestamp'] > "2018-08-06") & (df100['timestamp'] < "2018-11-07")].index

In [69]:
# 봄 1 여름 2 가을 3 겨울 0 
df100.iloc[sp_indice,-1] = 1
df100.iloc[sm_indice,-1] = 2
df100.iloc[fa_indice,-1] = 3

In [70]:
df100.columns

Index(['date', 'hour', 'humidity', 'rain', 'rain_type', 'temp', 'thunder',
       'wind_angle', 'wind_velo', 'sky_con', 'solar_radiation_normal', 'power',
       'date_time', 'timestamp', 'season'],
      dtype='object')

In [71]:
# 필요없는 column 제거 
df100 = df100[['date', 'hour', 'humidity', 'rain', 'rain_type', 'temp', 'thunder',
       'wind_angle', 'wind_velo', 'sky_con', 'solar_radiation_normal', 'season', 'power',
       'date_time']]

In [72]:
df100.head()

,date,hour,humidity,rain,rain_type,temp,thunder,wind_angle,wind_velo,sky_con,solar_radiation_normal,season,power,date_time
0,2018-01-01,6,17.0,0.0,0.0,4.7,0.0,289.0,2.8,1,0.00,0,0,2018-01-01-6
1,2018-01-01,7,19.0,0.0,0.0,3.4,0.0,278.0,2.7,1,0.00,0,0,2018-01-01-7
2,2018-01-01,8,14.0,0.0,0.0,2.7,0.0,268.0,2.3,1,0.00,0,7,2018-01-01-8
3,2018-01-01,9,18.0,0.0,0.0,2.3,0.0,268.0,2.3,2,4.78,0,245,2018-01-01-9
4,2018-01-01,10,20.0,0.0,0.0,2.1,0.0,250.0,3.0,2,17.79,0,776,2018-01-01-10


# 3. 미세먼지 데이터 붙이기
- donghae_solar_power_dataset2018_rev2.csv 데이터에 null값 직접 엑셀로 사이값 평균으로 채워 넣음

In [2]:
air = pd.read_csv('donghae_air2018.csv', index_col=0)
air.head()

,date,hour,SO2,CO,O3,NO2,PM10,PM25,date_time
0,2018-01-01,6,0.002,0.4,0.034,0.005,17.0,12.0,2018-01-01-6
1,2018-01-01,7,0.003,0.4,0.032,0.006,19.0,12.0,2018-01-01-7
2,2018-01-01,8,0.002,0.5,0.030,0.009,19.0,12.0,2018-01-01-8
3,2018-01-01,9,0.003,0.5,0.031,0.008,17.0,13.0,2018-01-01-9
4,2018-01-01,10,0.003,0.5,0.033,0.006,14.0,NaN,2018-01-01-10


In [74]:
air.columns

Index(['date', 'hour', 'SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25', 'date_time'], dtype='object')

In [75]:
air = air[['SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25', 'date_time']]
air.head()

,SO2,CO,O3,NO2,PM10,PM25,date_time
0,0.002,0.4,0.034,0.005,17.0,12.0,2018-01-01-6
1,0.003,0.4,0.032,0.006,19.0,12.0,2018-01-01-7
2,0.002,0.5,0.030,0.009,19.0,12.0,2018-01-01-8
3,0.003,0.5,0.031,0.008,17.0,13.0,2018-01-01-9
4,0.003,0.5,0.033,0.006,14.0,NaN,2018-01-01-10


In [76]:
df1000 = pd.merge(df100, air)
df1000.head()

,date,hour,humidity,rain,rain_type,temp,thunder,wind_angle,wind_velo,sky_con,solar_radiation_normal,season,power,date_time,SO2,CO,O3,NO2,PM10,PM25
0,2018-01-01,6,17.0,0.0,0.0,4.7,0.0,289.0,2.8,1,0.00,0,0,2018-01-01-6,0.002,0.4,0.034,0.005,17.0,12.0
1,2018-01-01,7,19.0,0.0,0.0,3.4,0.0,278.0,2.7,1,0.00,0,0,2018-01-01-7,0.003,0.4,0.032,0.006,19.0,12.0
2,2018-01-01,8,14.0,0.0,0.0,2.7,0.0,268.0,2.3,1,0.00,0,7,2018-01-01-8,0.002,0.5,0.030,0.009,19.0,12.0
3,2018-01-01,9,18.0,0.0,0.0,2.3,0.0,268.0,2.3,2,4.78,0,245,2018-01-01-9,0.003,0.5,0.031,0.008,17.0,13.0
4,2018-01-01,10,20.0,0.0,0.0,2.1,0.0,250.0,3.0,2,17.79,0,776,2018-01-01-10,0.003,0.5,0.033,0.006,14.0,NaN


In [77]:
df1000.columns

Index(['date', 'hour', 'humidity', 'rain', 'rain_type', 'temp', 'thunder',
       'wind_angle', 'wind_velo', 'sky_con', 'solar_radiation_normal',
       'season', 'power', 'date_time', 'SO2', 'CO', 'O3', 'NO2', 'PM10',
       'PM25'],
      dtype='object')

In [78]:
df1000 = df1000[['date', 'hour', 'humidity', 'rain', 'rain_type', 'temp', 'thunder',
       'wind_angle', 'wind_velo', 'sky_con', 'solar_radiation_normal',
       'season', 'SO2', 'CO', 'O3', 'NO2', 'PM10',
       'PM25','power','date_time']]
df1000.head()

,date,hour,humidity,rain,rain_type,temp,thunder,wind_angle,wind_velo,sky_con,solar_radiation_normal,season,SO2,CO,O3,NO2,PM10,PM25,power,date_time
0,2018-01-01,6,17.0,0.0,0.0,4.7,0.0,289.0,2.8,1,0.00,0,0.002,0.4,0.034,0.005,17.0,12.0,0,2018-01-01-6
1,2018-01-01,7,19.0,0.0,0.0,3.4,0.0,278.0,2.7,1,0.00,0,0.003,0.4,0.032,0.006,19.0,12.0,0,2018-01-01-7
2,2018-01-01,8,14.0,0.0,0.0,2.7,0.0,268.0,2.3,1,0.00,0,0.002,0.5,0.030,0.009,19.0,12.0,7,2018-01-01-8
3,2018-01-01,9,18.0,0.0,0.0,2.3,0.0,268.0,2.3,2,4.78,0,0.003,0.5,0.031,0.008,17.0,13.0,245,2018-01-01-9
4,2018-01-01,10,20.0,0.0,0.0,2.1,0.0,250.0,3.0,2,17.79,0,0.003,0.5,0.033,0.006,14.0,NaN,776,2018-01-01-10


In [79]:
# df1000.to_csv('donghae_solar_power_dataset2018_rev2.csv')